In [1]:
import sys
sys.path.append("/Users/ronald/Documents/code/harlus/server")


### Test installation

In [2]:
import harlus_doc_search
import harlus_chat
import harlus_contrast_tool


### Simulate app startup

In [3]:
import datetime
import json
import pickle
import time
from fastapi import (
    FastAPI,
    Query,
    Response,
    WebSocket,
)
import os
import asyncio
import nest_asyncio
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import FileResponse, JSONResponse, StreamingResponse
from openai import OpenAI
from platformdirs import user_data_dir
from pathlib import Path
from llama_index.core.agent.workflow import ReActAgent

from pydantic import BaseModel, ConfigDict, Field
from src.util import BoundingBoxConverter, snake_to_camel
from src.tool_library import ToolLibrary
from src.sync_workspace import get_workspace_sync_manager

# from src.stream_response import stream_generator_v2 # TODO: Delete this file
from src.file_store import FileStore, Workspace, File
from src.sync_queue import SyncQueue, SyncType
from src.stream_manager import StreamManager
from src.sync_status import SyncStatus
from src.chat_library import ChatLibrary
from harlus_contrast_tool import ContrastTool
from harlus_chat import ChatAgentGraph


app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=False,
    allow_methods=["*"],
    allow_headers=["*"],
)

asyncio.set_event_loop_policy(asyncio.DefaultEventLoopPolicy())


nest_asyncio.apply(asyncio)

DEV_APP_DATA_PATH = Path(user_data_dir("Electron")).joinpath("Harlus")

# On MacOs for development this will be something like
# /Users/danielglasgow/Library/Application Support/Electron
# TODO: Determine what this will be in production
# Probably something like /Users/danielglasgow/Library/Application Support/Harlus
# Also need to determine what this will be for Windows
# Ultimately this value is passed in by the Electron app in prod.
APP_DATA_PATH_STRING = os.environ.get("APP_DATA_PATH")
if (
    not APP_DATA_PATH_STRING
    or "Electron" in APP_DATA_PATH_STRING
    or "electron" in APP_DATA_PATH_STRING
):
    APP_DATA_PATH = DEV_APP_DATA_PATH
else:
    APP_DATA_PATH = Path(APP_DATA_PATH_STRING)

print("APP_DATA_PATH", APP_DATA_PATH)

file_store = FileStore(APP_DATA_PATH)

tool_library = ToolLibrary(file_store)
tool_library.load_tools()


# TODO: chat_library should call update_chat_tools when new tools
# are added to the tool_library to ensure chat has access to all tools 
# within the workspace. 
chat_library = ChatLibrary(file_store, tool_library)
chat_library.load_chats() # initialize a chat for each workspace, load from disk to memory


APP_DATA_PATH /Users/ronald/Library/Application Support/Electron/Harlus
Initializing FileStore with app_data_path /Users/ronald/Library/Application Support/Electron/Harlus


ModuleNotFoundError: No module named 'contrast_tool.mixed_retriever'